# This file is responsible for feature extraction in the SHU data, including embeddings, complexity measures, similarity measures and verb-argument constructions
## Import library

In [1]:
%run functions

c:\Users\GIGA\AppData\Local\Programs\Python\Python310\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


## Load data

In [2]:
loader = DataLoader()
d = loader.GetData('final')

## Clean text, remove redundant whitespace, \n, and 's

In [3]:
clean_text = []
preproc = preprocessing.make_pipeline(preprocessing.normalize.whitespace)
for essay in d['text']:
    essay = preproc(essay.replace('\n',''))
    punct = [':',',','.','?',';',"'",'!']
    for p in punct:
        essay = essay.replace(p,p+' ')
    for p in [' ' + i for i in punct]:
        essay = essay.replace(p,p.lstrip()) 
    cleaned = preproc(essay.replace('\n',''))
    clean_text.append(cleaned)
d['text'] = clean_text

## Get bert embeddings
21m 34.4s

In [ ]:
xa
for i in range(0,4000,1000):
    if i == 3000:
        end = len(d)
    else:
        end = i+1000
    EmbeddingFetcher = GetBERTEmbeddings(d['text'][i:end],'model/deberta-v3-large')
    EmbeddingFetcher.inf(stop=1000,SeqLen = 512)
    xp.append(EmbeddingFetcher.GetEmbeddings('MeanP')) 
    del EmbeddingFetcher
    torch.cuda.empty_cache()

## Save to disk
### Mean pooling

In [19]:
torch.save(xa,'features/final-meanp.pt')

## Extract fine-grained complexity features

135m56.1s

In [ ]:
for i in range(len(d)):
    t = d['text'][i]
    if len(t) > 2:
        extractor = FeatureExtraction(t)
        extractor.process()
        features = extractor.get_data()
        with open('features/final-winter-fine.csv','a',encoding='utf-8') as f:
            f.write('{},{},'.format(i,d['学号'][i]))
            f.write(','.join([str(i) for i in features]))
            f.write('\n')
    else:
        with open('features/final-winter-fine.csv','a',encoding='utf-8') as f:
            f.write('{},'.format(i))
            f.write('\n')
    print(i)

## Extract syntactic similarity

In [ ]:
sim_list = []
for num in range(len(d)):
    doc = NLP(d['text'][num])
    sents = [j for j in doc.sents if len([i for i in str(j).split() if i.isalpha()]) > 3]  
    self_sim = [1 for i in range(len(sents))] 
    sim_matrix= np.eye(len(sents),len(sents))
    for i in range(len(sents)):
        self_sim[i] = PartialTreeKernel(sents[i],sents[i])
        if self_sim[i] == 0:
            self_sim[i] = 1
    for i in range(len(sents)):
        for j in range(i+1,len(sents)):
            sim_matrix[i,j] = PartialTreeKernel(sents[i],sents[j])/math.sqrt(self_sim[i]*self_sim[j])
    #if num % 10 == 0:
    print(num)
    sim_list.append(sim_matrix)

In [6]:
np.savez('features/final_winter_sent.npz',*sim_list)

## Extract verb-vac

In [ ]:
vacs = []
for i in range(len(d)):
    doc = NLP(d['text'][i])
    doc_verb_vac = GetVAC(doc,render=False)
    for v in doc_verb_vac:
        if 'oprd' in v[0]:
            print(i,v)
    vacs.append(doc_verb_vac)
    if i % 50 == 0:
        print(i)

In [108]:
with open('features/final-verbvac.txt','w',encoding='utf-8') as f:
    for line in vacs:
        f.write((', '.join([i[0] for i in line])))
        f.write('\n')